In [1]:
import data_utils

In [2]:
path = "../data/absa/id/william/train.txt"
data = data_utils.read_data(path=path)

In [3]:
data[0]

{'text': 'kamar saya ada kendala di ac tidak berfungsi optimal . dan juga wifi koneksi kurang stabil .',
 'num_targets': [([5], [6, 7, 8], 'NEG'), ([12, 13], [14, 15], 'NEG')]}

In [4]:
ans_const = data_utils.AnswerConstructor()
ext_prompter = data_utils.ExtractPrompter()
imp_prompter = data_utils.ImputePrompter()
few_prompter = data_utils.FewShotPrompter()

In [5]:
from copy import deepcopy
nt = deepcopy(data[0]["num_targets"])
nt.append(deepcopy(nt[0]))
nt.append((deepcopy(nt[0][0]),deepcopy(nt[1][1]),deepcopy(nt[0][2])))
nt.append(([-1],deepcopy(nt[0][1]),"NEU"))
print(nt)

[([5], [6, 7, 8], 'NEG'), ([12, 13], [14, 15], 'NEG'), ([5], [6, 7, 8], 'NEG'), ([5], [14, 15], 'NEG'), ([-1], [6, 7, 8], 'NEU')]


# Answer Constructor

In [6]:
ans_const.lego_absa(data[0]["text"], nt, "aos", "oas")

'<extra_id_0> tidak berfungsi optimal <extra_id_1> ac <extra_id_2> negative ; <extra_id_3> kurang stabil <extra_id_4> wifi koneksi <extra_id_5> negative ; <extra_id_6> tidak berfungsi optimal <extra_id_7> ac <extra_id_8> negative ; <extra_id_9> kurang stabil <extra_id_10> ac <extra_id_11> negative ; <extra_id_12> tidak berfungsi optimal <extra_id_13> NULL <extra_id_14> neutral'

In [7]:
ans_const.gas(data[0]["text"], nt, "aos", "oas")

'( tidak berfungsi optimal , ac , negative ) ; ( kurang stabil , wifi koneksi , negative ) ; ( tidak berfungsi optimal , ac , negative ) ; ( kurang stabil , ac , negative ) ; ( tidak berfungsi optimal , NULL , neutral )'

In [8]:
ans_const.bartabsa(data[0]["text"], nt, "aos", "aos")

'5,5,6,8,NEG,12,13,14,15,NEG,5,5,6,8,NEG,5,5,14,15,NEG,-1,-1,6,8,NEU'

# Extraction Prompter

In [9]:
ext_prompter.lego_absa(data[0]["text"], "aos")

'kamar saya ada kendala di ac tidak berfungsi optimal . dan juga wifi koneksi kurang stabil .| aspect : <extra_id_0> ,opinion : <extra_id_1> ,sentiment : <extra_id_2>'

In [10]:
ext_prompter.gas(data[0]["text"], "aos")

'kamar saya ada kendala di ac tidak berfungsi optimal . dan juga wifi koneksi kurang stabil .| ( aspect , opinion , sentiment )'

In [11]:
ext_prompter.bartabsa(data[0]["text"], "aos")

'kamar saya ada kendala di ac tidak berfungsi optimal . dan juga wifi koneksi kurang stabil .| aspect_start,aspect_end,opinion_start,opinion_end,sentiment'

# Impute Prompter

In [12]:
imp_prompter.lego_absa(data[0]["text"], nt, "aos", "ao", "aos")

'kamar saya ada kendala di ac tidak berfungsi optimal . dan juga wifi koneksi kurang stabil .| aspect : ac ,opinion : tidak berfungsi optimal ,sentiment : <extra_id_0> ; aspect : wifi koneksi ,opinion : kurang stabil ,sentiment : <extra_id_1> ; aspect : ac ,opinion : kurang stabil ,sentiment : <extra_id_2> ; aspect : NULL ,opinion : tidak berfungsi optimal ,sentiment : <extra_id_3>'

In [13]:
imp_prompter.lego_absa(data[0]["text"], nt, "aos", "as", "aos")

'kamar saya ada kendala di ac tidak berfungsi optimal . dan juga wifi koneksi kurang stabil .| aspect : ac ,opinion : <extra_id_0> ,sentiment : negative ; aspect : wifi koneksi ,opinion : <extra_id_1> ,sentiment : negative ; aspect : NULL ,opinion : <extra_id_2> ,sentiment : neutral'

# Few shot prompter

In [14]:
few_prompter.lego_absa(data[0]["text"], data[0]["num_targets"], "aos", "aos", 1)

'kamar saya ada kendala di ac tidak berfungsi optimal . dan juga wifi koneksi kurang stabil .| aspect : wifi koneksi ,opinion : kurang stabil ,sentiment : negative ; aspect : <extra_id_0> ,opinion : <extra_id_1> ,sentiment : <extra_id_2>'

# Data Gen

In [15]:
tasks = [
    {
        "paradigm" : "extraction",
        "se_order" : "aos",
        "method" : "lego_absa"
    },
    {
        "paradigm" : "extraction",
        "se_order" : "ao",
        "method" : "lego_absa"
    },
    {
        "paradigm" : "imputation",
        "reduced_se_order" : "ao",
        "se_order" : "aos",
        "method" : "lego_absa"
    },
    {
        "paradigm" : "fewshot",
        "se_order" : "aos",
        "method" : "lego_absa",
        "n_shot" : 2
    }
]

result_ds = data_utils.data_gen(data,"aos",tasks,3,"round_robin")

100%|██████████| 9000/9000 [00:00<00:00, 39723.13it/s]


In [18]:
result_ds

[{'input': 'kamar saya ada kendala di ac tidak berfungsi optimal . dan juga wifi koneksi kurang stabil .| aspect : <extra_id_0> ,opinion : <extra_id_1> ,sentiment : <extra_id_2>',
  'output': '<extra_id_0> ac <extra_id_1> tidak berfungsi optimal <extra_id_2> negative ; <extra_id_3> wifi koneksi <extra_id_4> kurang stabil <extra_id_5> negative',
  'se_order': 'aos'},
 {'input': 'tempatnya bagus . kolam renangnya bersih .| aspect : <extra_id_0> ,opinion : <extra_id_1>',
  'output': '<extra_id_0> tempatnya <extra_id_1> bagus ; <extra_id_2> kolam renangnya <extra_id_3> bersih',
  'se_order': 'ao'},
 {'input': 'oke banget , tetapi ac nya tidak bisa diatur suhu nya .| aspect : ac nya ,opinion : tidak bisa diatur ,sentiment : <extra_id_0> ; aspect : NULL ,opinion : oke banget ,sentiment : <extra_id_1>',
  'output': '<extra_id_0> ac nya <extra_id_1> tidak bisa diatur <extra_id_2> negative ; <extra_id_3> NULL <extra_id_4> oke banget <extra_id_5> positive',
  'se_order': 'aos'},
 {'input': 'pint